In [1]:
# Clone repository จาก GitHub
!git clone https://github.com/Riririll/NGCF_SpecTopTermProj.git

Cloning into 'NGCF_SpecTopTermProj'...
remote: Enumerating objects: 398, done.
remote: Counting objects: 100% (173/173), done.
remote: Compressing objects: 100% (98/98), done.
remote: Total 398 (delta 76), reused 166 (delta 71), pack-reused 225 (from 1)
Receiving objects: 100% (398/398), 16.08 MiB | 15.36 MiB/s, done.
Resolving deltas: 100% (149/149), done.


In [2]:
import os
import pandas as pd
import numpy as np
import scipy.sparse as sp
import random

def filter_amazon_book(data_path, n_users=1000, n_items=7000, min_interactions=5):
    # อ่านไฟล์ train.txt
    with open(f"{data_path}/train.txt", 'r') as f:
        lines = f.readlines()

    # แปลงข้อมูลเป็น DataFrame
    interactions = []
    for line in lines:
        parts = line.strip().split()  # split() จะแยกตามช่องว่าง
        user = int(parts[0])          # user_id คือตัวแรก
        items = list(map(int, parts[1:]))  # item_ids คือรายการที่เหลือ
        for item in items:
            interactions.append([user, item])

    df = pd.DataFrame(interactions, columns=['user_id', 'item_id'])

    # นับจำนวน interactions ต่อ user และ item
    user_counts = df['user_id'].value_counts()
    item_counts = df['item_id'].value_counts()

    # เลือก users และ items ที่มี interactions มากที่สุด
    selected_users = user_counts[user_counts >= min_interactions].head(n_users).index
    selected_items = item_counts[item_counts >= min_interactions].head(n_items).index

    # กรองข้อมูลให้เฉพาะ users และ items ที่เลือก
    filtered_df = df[
        (df['user_id'].isin(selected_users)) &
        (df['item_id'].isin(selected_items))
    ]

    # สร้าง mapping ใหม่สำหรับ user_id และ item_id
    user_map = {old_id: new_id for new_id, old_id in enumerate(sorted(filtered_df['user_id'].unique()))}
    item_map = {old_id: new_id for new_id, old_id in enumerate(sorted(filtered_df['item_id'].unique()))}

    # แปลง IDs
    filtered_df['user_id'] = filtered_df['user_id'].map(user_map)
    filtered_df['item_id'] = filtered_df['item_id'].map(item_map)

    return filtered_df, user_map, item_map


def filter_test_file(test_file, user_map, item_map, output_test_file):
    # อ่านไฟล์ test.txt
    with open(test_file, 'r') as f:
        lines = f.readlines()

    test_interactions = []
    for line in lines:
        parts = line.strip().split()  # split() จะแยกตามช่องว่าง
        user = int(parts[0])          # user_id คือตัวแรก
        items = list(map(int, parts[1:]))  # item_ids คือรายการที่เหลือ
        for item in items:
            if user in user_map and item in item_map:  # กรองเฉพาะ user และ item ที่อยู่ใน mapping
                test_interactions.append([user_map[user], item_map[item]])

    # สร้าง DataFrame จากข้อมูลที่กรองแล้ว
    test_df = pd.DataFrame(test_interactions, columns=['user_id', 'item_id'])

    # บันทึกข้อมูลใน test.txt
    test_data = test_df.groupby('user_id')['item_id'].agg(list)
    with open(output_test_file, 'w') as f:
        for user_id, items in test_data.items():
            items_str = ' '.join(map(str, items))  # เปลี่ยนจากคอมม่าเป็นช่องว่าง
            f.write(f"{user_id} {items_str}\n")


# สร้างไฟล์ใหม่
def save_filtered_data(filtered_df, user_map, item_map, output_path):
    # สร้าง train.txt
    train_data = filtered_df.groupby('user_id')['item_id'].agg(list)
    with open(f"{output_path}/train.txt", 'w') as f:
        for user_id, items in train_data.items():
            items_str = ' '.join(map(str, items))  # เปลี่ยนจากคอมม่าเป็นช่องว่าง
            f.write(f"{user_id} {items_str}\n")

    # สร้าง user_list.txt
    with open(f"{output_path}/user_list.txt", 'w') as f:
        for old_id, new_id in user_map.items():
            f.write(f"{old_id} {new_id}\n")  # ใช้ช่องว่างเป็นตัวคั่น

    # สร้าง item_list.txt
    with open(f"{output_path}/item_list.txt", 'w') as f:
        for old_id, new_id in item_map.items():
            f.write(f"{old_id} {new_id}\n")  # ใช้ช่องว่างเป็นตัวคั่น

# กรองและบันทึกข้อมูล
data_path = "/content/NGCF_SpecTopTermProj/Data/amazon-book"
output_path = "/content/NGCF_SpecTopTermProj/Data/amazon-book-small"

# สร้างโฟลเดอร์ใหม่
os.makedirs(output_path, exist_ok=True)

# กรองข้อมูลใน train.txt
filtered_df, user_map, item_map = filter_amazon_book(data_path)
save_filtered_data(filtered_df, user_map, item_map, output_path)

# กรองและบันทึกข้อมูลใน test.txt
test_file = f"{data_path}/test.txt"
output_test_file = f"{output_path}/test.txt"
filter_test_file(test_file, user_map, item_map, output_test_file)

<ipython-input-2-83168eb7a93e>:42: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df['user_id'] = filtered_df['user_id'].map(user_map)
<ipython-input-2-83168eb7a93e>:43: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df['item_id'] = filtered_df['item_id'].map(item_map)


In [3]:
# อ่านข้อมูลจาก item_list.txt และ user_list.txt
item_map = {}
user_map = {}

# อ่าน item_list.txt (มีรูปแบบ (org_id, remap_id))
with open("/content/NGCF_SpecTopTermProj/Data/amazon-book-small/item_list.txt", "r") as f:
    for line in f.readlines():
        org_item_id, remap_item_id = map(int, line.strip().split())
        item_map[org_item_id] = remap_item_id

# อ่าน user_list.txt (มีรูปแบบ (org_id, remap_id))
with open("/content/NGCF_SpecTopTermProj/Data/amazon-book-small/user_list.txt", "r") as f:
    for line in f.readlines():
        org_user_id, remap_user_id = map(int, line.strip().split())
        user_map[org_user_id] = remap_user_id


In [4]:
# อ่านข้อมูลจาก train.txt และแปลงเป็น train.rating
train_data = []

with open("/content/NGCF_SpecTopTermProj/Data/amazon-book-small/train.txt", "r") as f:
    for line in f.readlines():
        parts = line.strip().split()

        # ID ใหม่ของ user และ item ที่ได้จาก train.txt
        remapped_user_id = int(parts[0])  # userID ใหม่จาก train.txt
        items = list(map(int, parts[1:]))  # itemIDs ใหม่จาก train.txt

        # สำหรับแต่ละ item ที่ผู้ใช้มีการปฏิสัมพันธ์ จะสร้าง triplet (user, item, rating=1)
        for item in items:
            train_data.append((remapped_user_id, item, 1))  # rating=1 สำหรับ positive interaction

# เขียนข้อมูลเป็น train.rating
with open("train.rating", "w") as f:
    for userID, itemID, rating in train_data:
        f.write(f"{userID}\t{itemID}\t{rating}\n")

print(f"Number of training instances: {len(train_data)}")

Number of training instances: 106055


In [5]:
# อ่านข้อมูลจาก test.txt และแปลงเป็น test.rating
test_data = []

with open("/content/NGCF_SpecTopTermProj/Data/amazon-book-small/test.txt", "r") as f:
    for line in f.readlines():
        parts = line.strip().split()

        # ID ใหม่ของ user และ item ที่ได้จาก test.txt
        remapped_user_id = int(parts[0])  # userID ใหม่จาก test.txt
        items = list(map(int, parts[1:]))  # itemIDs ใหม่จาก test.txt

        # สำหรับแต่ละ item ที่ผู้ใช้มีการปฏิสัมพันธ์ จะสร้าง triplet (user, item, rating=1)
        for item in items:
            test_data.append((remapped_user_id, item, 1))  # rating=1 สำหรับ positive interaction

# เขียนข้อมูลเป็น test.rating
with open("test.rating", "w") as f:
    for userID, itemID, rating in test_data:
        f.write(f"{userID}\t{itemID}\t{rating}\n")

print(f"Number of test instances: {len(test_data)}")

Number of test instances: 17306


In [6]:
import random

num_items = 7000  # จำนวนไอเท็มทั้งหมด

# ฟังก์ชันอ่านไฟล์ test.rating และสร้าง dictionary ที่เก็บ positive items
def load_test_rating(file_path):
    test_data = {}
    with open(file_path, 'r') as f:
        for line in f:
            parts = line.strip().split("\t")
            userID = int(parts[0])  # userID
            itemID = int(parts[1])  # itemID
            rating = float(parts[2])  # rating
            if userID not in test_data:
                test_data[userID] = set()
            test_data[userID].add(itemID)
    return test_data

# ฟังก์ชันสร้าง negative samples จาก positive items
def create_negative_samples(test_data, num_items):
    negative_samples = []
    for userID, pos_items in test_data.items():
        # สำหรับแต่ละ positive item (userID, itemID), สุ่ม negative items
        for itemID in pos_items:
            # สุ่มเลือก negative items ที่ไม่ได้มีการปฏิสัมพันธ์
            available_items = set(range(1, num_items + 1)) - pos_items
            negative_items = random.sample(available_items, 99)  # สุ่ม 99 negative items
            # สร้าง (userID, itemID) และตามด้วย 99 negative items
            negative_samples.append((userID, itemID, negative_items))
    return negative_samples

# ฟังก์ชันเขียน negative samples ลงในไฟล์ test.negative
def write_negative_samples(negative_samples, output_file):
    with open(output_file, "w") as f:
        for userID, itemID, neg_items in negative_samples:
            f.write(f"({userID},{itemID})\t" + "\t".join(map(str, neg_items)) + "\n")

input_file = "test.rating"  # ไฟล์ input test.rating
output_file = "test.negative"  # ไฟล์ output test.negative

# อ่านข้อมูลจาก test.rating
test_data = load_test_rating(input_file)

# สร้าง negative samples
negative_samples = create_negative_samples(test_data, num_items)

# เขียน negative samples ลงใน test.negative
write_negative_samples(negative_samples, output_file)

print(f"Number of negative samples generated: {len(negative_samples)}")



<ipython-input-6-dc3e1889e68d>:27: DeprecationWarning: Sampling from a set deprecated
since Python 3.9 and will be removed in a subsequent version.
  negative_items = random.sample(available_items, 99)  # สุ่ม 99 negative items


Number of negative samples generated: 17306
